In [1]:
import numpy as np
from ungol.wmd import wmd
from tqdm import tqdm_notebook as tqdm

f_db = '../opt/databases/180917_ungol.db.pickle'

TODO: loading config from ...ca/ungol-models/conf/logging.conf


In [2]:
from ungol.retrieval import clients
from ungol.retrieval import evaluate

In [3]:
un = clients.Ungol(f_db, multi=False, verbose=True, wmd_strategy=wmd.Strategy.ADAPTIVE_SMALL)
es = clients.Elasticsearch({'host': 'localhost', 'port': 9200})
client = clients.UngolReranking(es=es, un=un, preselect=250)

[clients] loading database into memory
[clients] loaded database:

VNGOL database
  containing: 294659 documents
  vocabulary: 400000 words
  filtering: 0 stopwords
  code size: 256 bits
  tokens: 258391
  skipped: 8

[common] connected to elasticsearch
[clients] initialized client


## General OOV

In [4]:
oov_counts = np.array([len(d.unknown) for d in un.db.mapping.values()])
print('mean', oov_counts.mean())
print('std', oov_counts.std())

mean 19.1258471657068
std 19.14673305268044


## Common OOV

In [5]:
from ungol.retrieval import experiment
from ungol.retrieval import stats

In [6]:
truth = experiment.Truth.from_file('../opt/src/task.clef2003.txt')

[experiment] read 21534 ground truth items
[experiment] imported 56 ground truth topics


In [7]:
# factory = stats.TopicStats.from_search_results

for top_id, task in tqdm(truth.items()):
    client.retrieve(top_id, k=250)

report = client.un.wmd_report

In [14]:
for top_id, top_report in report.items():
    for doc_id, doc_report in top_report.items():
        if len(doc_report.common_unknown):
            print(top_id, doc_id, doc_report.common_unknown)

141-AH FR940515-001799 {'pro7'}
141-AH FR941030-000195 {'pro7'}
141-AH FR940515-001735 {'pro7'}
141-AH FR941120-000169 {'pro7'}
141-AH FR940327-001661 {'pro7'}
141-AH FR941106-002187 {'pro7'}
141-AH FR940626-001496 {'pro7'}
141-AH FR940508-001473 {'pro7'}
141-AH FR940220-000134 {'pro7'}
141-AH FR940116-002112 {'pro7'}
141-AH FR940626-001567 {'pro7'}
141-AH FR941023-000707 {'pro7'}
141-AH FR940821-001670 {'pro7'}
141-AH FR940807-001510 {'pro7'}
141-AH FR940109-001415 {'pro7'}
141-AH FR940814-001398 {'pro7'}
141-AH SDA.940503.0057 {'pro7'}
141-AH FR940904-002665 {'pro7'}
141-AH FR940710-002112 {'pro7'}
141-AH FR940911-001134 {'pro7'}
145-AH SDA.940309.0158 {'reisimporte'}
145-AH SDA.940425.0201 {'reisimporte'}
145-AH FR940130-000652 {'reisimporte'}
145-AH SDA.940618.0045 {'reisimporte'}
149-AH SDA.950117.0108 {'papstbesuches'}
149-AH SDA.950116.0107 {'papstbesuches'}
159-AH FR940130-000679 {'ölsuche'}
161-AH SDA.941006.0058 {'ernährungsprobleme'}
165-AH SDA.950122.0072 {'1994'}
165-AH SD